
# Latest API data
This notebook consists of 3 parts:
1. Get new data from the API
2. Data preprocessing and feature engineering
3. Creating or backfilling the feature group

In [1]:
# Standard library imports for utilities and data manipulation
import uuid
import os
from datetime import datetime, timedelta
import io
import base64

# External libraries for data handling, networking, and time zones
import pandas as pd
import requests
import json
import pytz

# Environment variable management
from dotenv import load_dotenv
load_dotenv()

# Hopsworks
import hopsworks                

## 1. Get new data from the API

### Sensor Data Access

Here is the information given by the company so we can acces thir data.

- GET request to `data.sensade.com` 
- Authentication: `Basic Auth` (user: miknie20@student.aau.dk, password: GitHub Secret)

### Sensors

Two sensors installed:

- `0080E115003BEA91` (Hw2.0 Fw2.0) Installed towards building

- `0080E115003E3597` (Hw2.0 Fw2.0) Installed towards bike lane

In [52]:
# Create a timezone object for GMT+2
timezone = pytz.timezone('Europe/Copenhagen')
now = datetime.now(timezone)  # Get current time 
today = now 
yesterday = today - timedelta(days=1)
tomorrow = today + timedelta(days=1)
#Defining two hours ago
two_hours_ago = now - timedelta(hours=2)

In [53]:
# Format 'today', 'tomorrow', and 'yesterday' as "YYYY-MM-DD"
formatted_today = today.strftime('%Y-%m-%d %H:%M:%S')
formatted_tomorrow = tomorrow.strftime('%Y-%m-%d %H:%M:%S')
formatted_yesterday = yesterday.strftime('%Y-%m-%d %H:%M:%S')
formatted_two_hours_ago = two_hours_ago.strftime('%Y-%m-%d %H:%M:%S')
dev_eui_building = "0080E115003BEA91"
dev_eui_bikelane = "0080E115003E3597"
url = "https://data.sensade.com"

basic_auth = base64.b64encode(f"{os.getenv('API_USER')}:{os.getenv('API_PASSWORD')}".encode())
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Basic {basic_auth.decode("utf-8")}'
}

In [50]:
# API call that takes the data from given sensor from yesterday until tomorrow
def API_call(dev_eui, from_date, to_date):
    payload = json.dumps({
    "dev_eui": dev_eui,
    "from": from_date,
    "to": to_date
})

    response = requests.request("GET", url, headers=headers, data=payload)

    if response.status_code != 200:
     print("Failed to fetch data: Status code", response.status_code)     
     print("Response:", response.text)     
     exit(13)

    csv_data = response.text
    df = pd.read_csv(io.StringIO(csv_data))
    return df

In [55]:
# Running the API call function with the given parameters on the building sensor
df_building_from_api = API_call(dev_eui_building, formatted_yesterday, formatted_tomorrow)

In [56]:
# Running the API call function with the given parameters on the bikelane sensor
df_bikelane_from_api = API_call(dev_eui_bikelane, formatted_yesterday, formatted_tomorrow)

In [57]:
# Defning the newest data from the API calls
df_building_newest = df_building_from_api.tail(1)
df_bikelane_newest = df_bikelane_from_api.tail(1)

In [58]:
df_building_newest

,time,battery,temperature,x,y,z,0_radar,1_radar,2_radar,3_radar,4_radar,5_radar,6_radar,7_radar,package_type,f_cnt,dr,snr,rssi,hw_fw_version
121,2024-05-17 07:56:02.483000,3.2,16.875,388,233,65,25.0,44.0,43.0,44.0,44.0,28.0,14.0,8.0,PackageType.HEART_BEAT,5510,5,-3.8,-89,DataVersion.HW_2FW2_X_X


In [59]:
df_bikelane_newest

,time,battery,temperature,x,y,z,0_radar,1_radar,2_radar,3_radar,4_radar,5_radar,6_radar,7_radar,package_type,f_cnt,dr,snr,rssi,hw_fw_version
128,2024-05-17 07:37:16.564000,3.1,14.625,-233,158,-388,44.0,63.0,67.0,88.0,51.0,36.0,29.0,14.0,PackageType.HEART_BEAT,5382,5,0.2,-89,DataVersion.HW_2FW2_X_X


## 2. Preprocessing and feature engineering

We apply the same methods as in notebook 1: creating unique IDs, converting the time column, converting radar names, changing data types to floats, and finally creating an empty column for the mag_cluster, as we haven't applied our models yet.

In [8]:
# Create a unique identifier for each row in the datasets
def create_id(df, dataset_name):
    # Assign the sensor prefix based on the dataset name
    if dataset_name == 'df_building_newest':
        df['psensor'] = "BUILDING"
    elif dataset_name == 'df_bikelane_newest':
        df['psensor'] = "BIKELANE"
    else:
        raise ValueError("Unknown dataset name provided")

    # Create a new column 'id' with a unique identifier for each row
    df['id'] = [str(uuid.uuid4()) for _ in df.index]

    return df

In [9]:
# Applying the function to the datasets
df_bikelane_newest = df_bikelane_newest.copy()
df_bikelane = create_id(df_bikelane_newest, 'df_bikelane_newest')
df_building_newest = df_building_newest.copy()
df_building = create_id(df_building_newest, 'df_building_newest')

In [10]:
#converting the time column to datetime
df_bikelane['time'] = pd.to_datetime(df_bikelane['time'])
df_building['time'] = pd.to_datetime(df_building['time'])

In [11]:
#Renaming the radar columns to start with radar
df_bikelane = df_bikelane.rename(columns={'0_radar': 'radar_0', '1_radar': 'radar_1', '2_radar': 'radar_2', '3_radar': 'radar_3', '4_radar': 'radar_4', '5_radar': 'radar_5', '6_radar': 'radar_6', '7_radar': 'radar_7'})
df_building = df_building.rename(columns={'0_radar': 'radar_0', '1_radar': 'radar_1', '2_radar': 'radar_2', '3_radar': 'radar_3', '4_radar': 'radar_4', '5_radar': 'radar_5', '6_radar': 'radar_6', '7_radar': 'radar_7'})


In [12]:
# Converting the columns to float
df_bikelane[['x','y','z', 'radar_0', 'radar_1', 'radar_2', 'radar_3', 'radar_4', 'radar_5', 'radar_6', 'radar_7', 'f_cnt', 'dr', 'rssi']] = df_bikelane[['x','y','z', 'radar_0', 'radar_1', 'radar_2', 'radar_3', 'radar_4', 'radar_5', 'radar_6', 'radar_7', 'f_cnt', 'dr', 'rssi']].astype(float)
df_building[['x','y','z', 'radar_0', 'radar_1', 'radar_2', 'radar_3', 'radar_4', 'radar_5', 'radar_6', 'radar_7', 'f_cnt', 'dr', 'rssi']] = df_building[['x','y','z', 'radar_0', 'radar_1', 'radar_2', 'radar_3', 'radar_4', 'radar_5', 'radar_6', 'radar_7', 'f_cnt', 'dr', 'rssi']].astype(float)


In [13]:
#making an empty label column
df_bikelane['mag_cluster'] = "null"
df_building['mag_cluster'] = "null"

## Backfill or create feature group

In [26]:
# Connceting to the Hopsworks project

project = hopsworks.login(project="annikaij")

fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549019
Connected. Call `.close()` to terminate connection gracefully.


In [15]:
bikelane_fg = fs.get_or_create_feature_group(name="api_bikelane_newest",
                                  version=1,
                                  primary_key=["id"],
                                  event_time='time',
                                  description="New bike lane data",
                                  online_enabled=True,
                                 )
bikelane_fg.insert(df_bikelane)

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: api_bikelane_newest_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549014/jobs/named/api_bikelane_newest_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x759c599f7550>, None)

In [16]:
building_fg = fs.get_or_create_feature_group(name="api_building_newest",
                                    version=1,
                                    primary_key=["id"],
                                    event_time='time',
                                    description="New building data",
                                    online_enabled=True
                                     )
building_fg.insert(df_building)

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: api_building_newest_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549014/jobs/named/api_building_newest_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x759c58e5f250>, None)

## **Next up:** 3: Feature view creation
Go to the 3_featureview_creation.ipynb notebook